In [1]:
import os

data_path = './../dataset'

classes = [
    'Cyclone', 
    'Earthquake', 
    'Flood', 
    'Wildfire'
]

In [2]:
train_split = 0.80            #80% from total dataset
validation_split = 0.125      #12.5% from training split
test_split = 0.20             #20% from total dataset

In [3]:
#some hyper parameters
batch_size = 32
step_size = 8
number_of_epochs = 48

In [ ]:
import time

# import matplotlib to draw figure
import matplotlib
matplotlib.use("Agg")

# importing necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, Dense, Flatten, Add, AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNetRS152
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import GlobalAveragePooling2D
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import sys

print('All dependencies are added')

In [1]:
print("Loading all images path")
imagePaths = list(paths.list_images(data_path))
data = []
labels = []
print("Done!!")
print("Loaded ", len(imagePaths), "images path")

Loading all images path


NameError: name 'paths' is not defined

In [6]:
# loop over the image paths
for imagePath in imagePaths:
    # extract the class label
    label = imagePath.split(os.path.sep)[-2]
    # load the image, convert it to RGB channel ordering, and resize
    # it to be a fixed 224x224 pixels, ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)
# convert the data and labels to NumPy arrays
print("[INFO] processing data...")
data = np.array(data, dtype="float32")
labels = np.array(labels)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

print("All images labeled")

[INFO] processing data...
All images labeled


In [8]:
# partition the data into training and testing splits
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=test_split, random_state=31)

In [9]:
# take the validation split from the training split
(trainX, valX, trainY, valY) = train_test_split(trainX, trainY, test_size=validation_split, random_state=62)

In [17]:
# Load the pre-trained ResNet model
resnet = ResNetRS152(weights='imagenet', include_top=False)

# Freeze the weights of the ResNet model
for layer in resnet.layers:
    layer.trainable = False

# Add custom layers on top of the ResNet model
num_classes = len(classes)
x = resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=resnet.input, outputs=predictions)

In [18]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
# Train the model on your data
model.fit(trainX, trainY, epochs=number_of_epochs, batch_size = batch_size, validation_data=(valX, valY))

Epoch 1/48
97/97 [==============================] - 226s 2s/step - loss: 0.2232 - accuracy: 0.9248 - val_loss: 0.1217 - val_accuracy: 0.9661
Epoch 2/48
97/97 [==============================] - 222s 2s/step - loss: 0.0778 - accuracy: 0.9745 - val_loss: 0.1060 - val_accuracy: 0.9684
Epoch 3/48
97/97 [==============================] - 228s 2s/step - loss: 0.0605 - accuracy: 0.9787 - val_loss: 0.1423 - val_accuracy: 0.9571
Epoch 4/48
97/97 [==============================] - 223s 2s/step - loss: 0.0484 - accuracy: 0.9823 - val_loss: 0.1294 - val_accuracy: 0.9594
Epoch 5/48
97/97 [==============================] - 224s 2s/step - loss: 0.0304 - accuracy: 0.9890 - val_loss: 0.1137 - val_accuracy: 0.9707
Epoch 6/48
97/97 [==============================] - 222s 2s/step - loss: 0.0201 - accuracy: 0.9939 - val_loss: 0.1123 - val_accuracy: 0.9639
Epoch 7/48
97/97 [==============================] - 222s 2s/step - loss: 0.0208 - accuracy: 0.9926 - val_loss: 0.1415 - val_accuracy: 0.9616
Epoch 8/48
97

In [21]:
model.save("model_v3.h5")

In [22]:
# Evaluate the model on your test data
loss, accuracy = model.evaluate(testX, testY)
print('Accuray: ', accuracy)
print('Loss: ', loss)

28/28 [==============================] - 50s 2s/step - loss: 0.3966 - accuracy: 0.9549
Accuray:  0.9548532962799072
Loss:  0.39656201004981995
